# Routes and Mileage

## Extract routes and endpoints from static MTA data

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import numpy as np

In [2]:
routes = pd.Series([], dtype=str)
endpoints_1 = pd.Series([], dtype=str)
endpoints_2 = pd.Series([], dtype=str)

boroughs = ["google_transit_bronx", "google_transit_brooklyn", "google_transit_manhattan", "google_transit_queens",
           "google_transit_staten_island"]

for borough in boroughs:
    filename = f"mta_bus_data/{borough}/routes.txt"
    df = pd.read_csv(filename)
    routes = routes.append(df["route_short_name"], ignore_index=True)
    start_finish_df = df["route_long_name"].str.split(" - ", expand=True)
    endpoints_1 = endpoints_1.append(start_finish_df[0], ignore_index=True)
    endpoints_2 = endpoints_2.append(start_finish_df[1], ignore_index=True)

routes_df = pd.DataFrame()
routes_df["Route"] = routes
routes_df["End point 1"] = endpoints_1
routes_df["End point 2"] = endpoints_2
routes_df = routes_df.set_index("Route")

In [3]:
routes_df.head()

,End point 1,End point 2
Route,,
Bx1,Riverdale,Mott Haven
Bx10,Riverdale,Norwood
Bx11,Parkchester,Southern Blvd & GW Bridge
Bx12,Pelham Pkwy,Fordham Rd
Bx12-SBS,Pelham Pkwy,Fordham Rd


## Get bus stops in every route

In [11]:
BUSTIME_URL = "https://bustime.mta.info"

stops_df = pd.DataFrame(columns=["Route", "Direction"])
stops_df = stops_df.set_index(["Route", "Direction"])

for route in tqdm(routes):
    query = f"{BUSTIME_URL}/m/index?q={route}"
    html = requests.get(query).text
    soup = BeautifulSoup(html, "html5lib")
    for div in soup.findAll("div", {"class": "directionForRoute"}):
        route_direction = div.find("p", {"class": "directionTitle"}).getText()
        route, _, direction  = route_direction.split(maxsplit=2)
        stop_num = 1
        for stop in div.findAll("li", {"class": ["start", "middle", "end"]}):
                stop_name = stop.find("a").getText()
                stops_df.loc[(route, direction), f"Stop {stop_num}"] = stop_name
                stop_num += 1

100%|██████████| 242/242 [02:49<00:00,  1.43it/s]


In [12]:
stops_df.replace(np.nan, "", inplace=True)
stops_df.dropna(axis=1, how="all", inplace=True)
stops_df.head()

Stop 1  \
Route Direction                                                    
Bx1   MOTT HAVEN 138 ST via CONCOURSE      RIVERDALE AV/W 231 ST   
      RIVERDALE 231 ST via CONCOURSE         LINCOLN AV/E 137 ST   
Bx10  NORWOOD 205 ST STA                   RIVERDALE AV/W 263 ST   
      RIVERDALE 263 ST                   E 206 ST/ROCHAMBEAU AVE   
Bx11  WASHINGTON HEIGHTS G W BRIDGE    W FARMS RD/WESTCHESTER AV   

                                                                  Stop 2  \
Route Direction                                                            
Bx1   MOTT HAVEN 138 ST via CONCOURSE                W 231 ST/CORLEAR AV   
      RIVERDALE 231 ST via CONCOURSE            GRAND CONCOURSE/E 138 ST   
Bx10  NORWOOD 205 ST STA                           RIVERDALE AV/W 261 ST   
      RIVERDALE 263 ST                 E MOSHOLU PY N/VAN CORTLANDT AV E   
Bx11  WASHINGTON HEIGHTS G W BRIDGE                  W FARMS RD/E 167 ST   

                                                                 Stop 3  \
Route Direction                                                           
Bx1   MOTT HAVEN 138 ST via CONCOURSE                 W 231 ST/BROADWAY   
      RIVERDALE 231 ST via CONCOURSE           GRAND CONCOURSE/E 144 ST   
Bx10  NORWOOD 205 ST STA                          RIVERDALE AV/W 260 ST   
      RIVERDALE 263 ST                 VAN CORTLANDT AV E/ROCHAMBEAU AV   
Bx11  WASHINGTON HEIGHTS G W BRIDGE            W FARMS RD/LONGFELLOW AV   

                                                               Stop 4  \
Route Direction                                                         
Bx1   MOTT HAVEN 138 ST via CONCOURSE              W 231 ST/BAILEY AV   
      RIVERDALE 231 ST via CONCOURSE         GRAND CONCOURSE/E 149 ST   
Bx10  NORWOOD 205 ST STA                        RIVERDALE AV/W 259 ST   
      RIVERDALE 263 ST                 BAINBRIDGE AV/RESERVOIR OVAL E   
Bx11  WASHINGTON HEIGHTS G W BRIDGE            E 172 ST/LONGFELLOW AV   

                                                         Stop 5  \
Route Direction                                                   
Bx1   MOTT HAVEN 138 ST via CONCOURSE      HEATH AV/ALBANY CRES   
      RIVERDALE 231 ST via CONCOURSE   GRAND CONCOURSE/E 153 ST   
Bx10  NORWOOD 205 ST STA                  RIVERDALE AV/W 256 ST   
      RIVERDALE 263 ST                   BAINBRIDGE AV/E 210 ST   
Bx11  WASHINGTON HEIGHTS G W BRIDGE            E 172 ST/VYSE AV   

                                                            Stop 6  \
Route Direction                                                      
Bx1   MOTT HAVEN 138 ST via CONCOURSE           HEATH AV/SUMMIT PL   
      RIVERDALE 231 ST via CONCOURSE      GRAND CONCOURSE/E 156 ST   
Bx10  NORWOOD 205 ST STA                     RIVERDALE AV/W 254 ST   
      RIVERDALE 263 ST                 E GUN HILL RD/BAINBRIDGE AV   
Bx11  WASHINGTON HEIGHTS G W BRIDGE           E 172 ST/SOUTHERN BL   

                                                               Stop 7  \
Route Direction                                                         
Bx1   MOTT HAVEN 138 ST via CONCOURSE  SEDGWICK AV/FT INDEPENDENCE ST   
      RIVERDALE 231 ST via CONCOURSE         GRAND CONCOURSE/E 161 ST   
Bx10  NORWOOD 205 ST STA                  HENRY HUDSON PKY W/W 252 ST   
      RIVERDALE 263 ST                       E GUN HILL RD/DE KALB AV   
Bx11  WASHINGTON HEIGHTS G W BRIDGE            LOUIS NINE BL/E 170 ST   

                                                            Stop 8  \
Route Direction                                                      
Bx1   MOTT HAVEN 138 ST via CONCOURSE         SEDGWICK AV/GILES PL   
      RIVERDALE 231 ST via CONCOURSE      GRAND CONCOURSE/E 163 ST   
Bx10  NORWOOD 205 ST STA               HENRY HUDSON PKY W/W 249 ST   
      RIVERDALE 263 ST                    JEROME AV/E MOSHOLU PY N   
Bx11  WASHINGTON HEIGHTS G W BRIDGE        LOUIS NINE BL/BOSTON RD   

                                              

In [13]:
OUTPUT_FILE_STOPS = "bustime_stops/stops_by_route.csv"
stops_df.to_csv(OUTPUT_FILE_STOPS)